# Simulating a simple polarimeter
In this tutorial we will simulate with hcipy a simple polarimeter that measures linear polarization states. It will consist of a rotating half-wave plate (HWP; the polarization modulator) and a linear polarizer (the analyzer). The system will be a single-beam, which means that it can only measure one polarization state at the time. Therefore, it will use temporal polarization modulation (i.e. the rotating HWP) to measure the other states as well. We will use this polarimeter to measure the polarization state of a star. 

A good introduction on polarimetry can be found in Snik, F., & Keller, C. U. (2013). Astronomical polarimetry: polarized views of stars and planets. We assume a basic knowledge on polarimetry (e.g. Stokes vectors, waveplates, polarizers). 

We start by importing the relevant python modules. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

from hcipy import * 

We will do our measurements with a 4 meter diameter telescope at a wavelength of 500 nanometer. This will set the spatial resolution of the system. 

In [ ]:
# parameters telescope
telescope_diameter = 4 # meter 
wavelength = 500E-9 #meter

# the spatial resolution
spatial_resolution_telescope = wavelength / telescope_diameter

This allows us to set the pupil- and focal-plane grids, and the propagator.

In [ ]:
# setting the grids
pupil_grid = make_pupil_grid(128, telescope_diameter)
focal_grid = make_focal_grid(q=6, num_airy=10, spatial_resolution=spatial_resolution_telescope)

# the propagator between the pupil- and focal-grid. 
propagator = FraunhoferPropagator(pupil_grid, focal_grid) 

Defining the aperture, the HWP positions, the linear polarizer and the detector. This detector is perfect in the sense that it has no dark current, no read noise and no flat field errors. Therefore, we will only suffer from photon noise. 

In [ ]:
aperture = circular_aperture(telescope_diameter)(pupil_grid)

# states of the half-wave plate. 
HWP_position_1 = HalfWavePlate(0)
HWP_position_2 = HalfWavePlate(np.radians(45))
HWP_position_3 = HalfWavePlate(np.radians(22.5))
HWP_position_4 = HalfWavePlate(np.radians(67.5))

polarizer = LinearPolarizer(0)

detector = NoisyDetector(focal_grid, dark_current_rate=0, read_noise=0, flat_field=0, include_photon_noise=False)

We now define the parameters for the starlight, e.g. its flux level and polarization state, which we eventually hope to measure.

In [ ]:
# parameters star
zero_magnitude_flux = 3.9E10
stellar_magnitude = 8

# the polarization state of the starlight.
stokes_vector_star = np.array([1,0.05,-0.2,0.1])

# Here we give the wavefront the properties (power, polarization state) of the starlight.
pupil_wavefront = Wavefront(aperture, wavelength, input_stokes_vector=stokes_vector_star)

pupil_wavefront.total_power = zero_magnitude_flux * 10**(-stellar_magnitude / 2.5)

print("Total photon flux {:g} photons / sec.".format(pupil_wavefront.total_power))

We can check the polarization state of the wavefront simply by: 

In [ ]:
stokes_parameters = [pupil_wavefront.I, pupil_wavefront.Q, pupil_wavefront.U, pupil_wavefront.V]
titles = ['I', 'Q', 'U', 'V']

# The value that we use to normalize the stokes vector with. 
max_val = np.max(stokes_parameters[0])

k=1
plt.figure(figsize=(16, 8))

for stokes_parameter, title in zip(stokes_parameters, titles):
        
    if max_val == 0:
        max_val = 1

    plt.subplot(1,4,k)

    imshow_field(stokes_parameter / max_val, cmap='bwr', vmin=-1, vmax=1)
    
    plt.xlabel('x [meter]')
    
    if title == 'I':
        plt.ylabel('y [meter]')
    
    plt.title('Stokes ' + title)
    k += 1

We will now simulate our polarimeter for a given time duration. During the simulation we will perform multiple HWP cycles. During one HWP cycle the HWP will rotate through its four positions. For every HWP position we will make an intensity measurement.   

In [ ]:

# total duration of the measurement
measurement_duration = 8 # seconds

# number of times we go through a HWP cycle
HWP_cycles = 4

# integration time per measurement
delta_t = measurement_duration / (HWP_cycles * 4) 

# counter for the state of the modulation loop 
k = 0 

# The arrays where the measurements are saved 
measurements = Field(np.zeros((4, focal_grid.size)), focal_grid)

# looping through the time steps 
for t in np.linspace(0, measurement_duration, HWP_cycles * 4):
    
    # selecting the HWP position
    if k == 0:
        HWP_position = HWP_position_1
    elif k == 1:
        HWP_position = HWP_position_2
    elif k == 2:
        HWP_position = HWP_position_3
    elif k == 3:
        HWP_position = HWP_position_4
        
    # we propagate the wavefront through the half-wave plate 
    pupil_wavefront_2 = HWP_position.forward(pupil_wavefront)
    
    # we propagate the wavefront through the linear polarizer 
    pupil_wavefront_3 = polarizer.forward(pupil_wavefront_2)
    
    focal_wavefront = propagator(pupil_wavefront_3)

    detector.integrate(focal_wavefront, dt=delta_t)
    
    # reading out the detector in the correct element of the measurement array
    measurements[k,:] += detector.read_out()
    
    # Moving to the next HWP position
    k += 1 

    # resetting the HWP to its intial position
    if k > 3:
        k = 0

Lets plot the measurements for the various HWP positions and the total number of photons. Note that the measurements have different numbers of photons, this is due to the starlight's polarization state. 

In [ ]:
plt.figure(figsize=(16, 8))

max_val_meas = np.max(measurements) 

for i in np.arange(4):
    plt.subplot(1,4,i+1)
    imshow_field(np.log10(measurements[i,:] / max_val_meas), vmin=-4, vmax=0)
    
    plt.xlabel('x [rad]')
    
    if i == 0:
        plt.ylabel('y [rad]')
    
    print('\nHWP position ', i+1)
    print('Number of photons = ', int(np.sum(measurements[i,:])))

    plt.title('HWP position ' + str(i+1))

We now have our measurements, which we want to convert into a polarization state. We do this by multiplying the measurements with a demodulation matrix. This matrix combines the measurements in such a way that the polarization state is retrieved. The demodulation matrix for this system is given by:

In [ ]:
# defining the demodulation matrix
demodulation_matrix = np.zeros((4,4))

# demodulation for I
demodulation_matrix[0,:] = 0.25

# demodulation for Q
demodulation_matrix[1,0] = 0.5
demodulation_matrix[1,1] = -0.5

# demodulation for U
demodulation_matrix[2,2] = 0.5
demodulation_matrix[2,3] = -0.5

print('demodulation matrix = \n', demodulation_matrix)

Let's do aperture photometry on the star to construct a 1-dimensional measurement vector. We use an aperture with a diameter of the spatial resolution of the telescope (i.e. $1$ $\lambda/D$). 

After that, we calculate the polarization state by multiplying this vector with the demodulation matrix. 

We see that we can completely retrieve the linear polarization state, but that we are not able to measure circular polarization.

In [ ]:
photometry_aperture = circular_aperture(spatial_resolution_telescope)(focal_grid)

# generating the measurement vector by doing aperture photometry. 
measurement_vector = np.array(np.sum(measurements[:,photometry_aperture==1], axis=1))

# calculating the measured Stokes vector 
stokes_measured = field_dot(demodulation_matrix, measurement_vector)

print('Measured Stokes vector = \n', stokes_measured / stokes_measured[0])

print('Input Stokes vector = \n', pupil_wavefront.input_stokes_vector)


To make this simulation more realistic, one can add the following:
1. realistic telescope aperture
2. non-perfect polarization optics
3. atmospheric turbulence and adaptive optics
4. detector noise (e.g. dark current, read noise, etc)